In [1]:
print('ok')

ok


In [1]:
%pwd

'c:\\Users\\sukan\\OneDrive\\Generative_AI\\GenAI_chat_bot\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\sukan\\OneDrive\\Generative_AI\\GenAI_chat_bot'

In [4]:
from langchain.document_loaders import  PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def load_pdf_data(data):
    loader=DirectoryLoader(data,glob='*.pdf',loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
%pwd

'c:\\Users\\sukan\\OneDrive\\Generative_AI\\GenAI_chat_bot'

In [7]:
extracted_data=load_pdf_data(data='Data/')

In [8]:
# extracted_data
def text_split(extracted_data):
    text_spliter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_spliter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print('length of chunks',len(text_chunks))

length of chunks 5699


In [10]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [11]:
def download_hugging_face_embeddings():
    embeddngs=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddngs

In [12]:
embeddngs=download_hugging_face_embeddings()

C:\Users\sukan\AppData\Local\Temp\ipykernel_12400\339533115.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddngs=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\sukan\OneDrive\Generative_AI\GenAI_chat_bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
print(embeddngs)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} query_instruction='Represent this question for searching relevant passages: ' embed_instruction='' show_progress=False


In [14]:
results=embeddngs.embed_query('Hello world')
print('length of results',len(results))

length of results 384


In [15]:
results

[-0.010300828143954277,
 0.18307934701442719,
 0.030811281874775887,
 0.004452868364751339,
 -0.027336159721016884,
 -0.0335625596344471,
 0.03763158619403839,
 -0.03157338127493858,
 -0.0033909634221345186,
 -0.008950830437242985,
 0.03803609684109688,
 -0.051291048526763916,
 0.00036826470750384033,
 -0.02372710593044758,
 0.09271029382944107,
 -0.02779584378004074,
 -0.03515247628092766,
 -0.0032241714652627707,
 -0.0768178328871727,
 -0.057612095028162,
 0.07257598638534546,
 0.11128553748130798,
 0.01605852320790291,
 0.01590849459171295,
 -0.0823269784450531,
 0.007007323205471039,
 0.029013115912675858,
 0.0011386671103537083,
 0.11671741306781769,
 -0.032327406108379364,
 -0.03227165341377258,
 -0.0012590623227879405,
 0.10591620206832886,
 0.023600773885846138,
 0.00966486893594265,
 0.09834077209234238,
 0.04293639212846756,
 -0.019547687843441963,
 0.01926792599260807,
 -0.06417104601860046,
 0.023923387750983238,
 -0.05287997052073479,
 -0.026469580829143524,
 0.00554872630

In [92]:
from dotenv import load_dotenv
load_dotenv()

True

In [93]:
PINECONE_API_KEY=os.environ.get('PINE_CONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
Groq_API_KEY=os.environ.get('Groq_API_KEY')

In [46]:
# from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
import os
pc = Pinecone(api_key=PINECONE_API_KEY)

In [56]:
index_name = "medicalchatbot2"
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
# pc.create_index_for_model(
#     name=index_name,
#     dimension=384,
#     metric='cosine',
#     spec=ServerlessSpec(cloud='aws',
#                         region='us-east-1')
# )

# if not pc.has_index(index_name):
#     pc.create_index_for_model(
#         name=index_name,
#         cloud="aws",
#         region="us-east-1",
#         embed={
#             "model":"llama-text-embed-v2",
#             "field_map":{"text": "chunk_text"}
#         }
#     )

{
    "name": "medicalchatbot2",
    "metric": "cosine",
    "host": "medicalchatbot2-m0n4dbo.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [94]:
import os
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
os.environ['Groq_API_KEY']=Groq_API_KEY


# Groq_API_KEY=os.environ.get('Groq_API_KEY')

In [58]:
from langchain_pinecone import PineconeVectorStore
doc_search=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddngs
    

)

In [59]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    # documents=text_chunks,
    index_name=index_name,
    embedding=embeddngs)

In [60]:
docsearch

In [61]:
retriever=docsearch.as_retriever(search_type='similarity',search_kwars={"k":3})

In [76]:
retrieved_docs=retriever.invoke('what is Acne')

In [77]:
retrieved_docs

[Document(id='45bbfbb5-4133-4880-b373-cf0f9c1cdfb2', metadata={'author': 'Clifford', 'creationdate': '2004-12-28T15:38:25-05:00', 'creator': 'PyPDF', 'enhanced': 'By PDF Enhancer 2.5/Win', 'moddate': '2005-05-04T13:53:15-06:00', 'page': 4.0, 'page_label': '5', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'spdf': '1096', 'total_pages': 599.0}, page_content='A\nAbscess\nAcidophilus\nAcne\nAconite\nAcupressure\nAcupuncture\nAdemetionine\nAdie’s pupil\nAfrican pygeum\nAgastache\nAging\nAIDS\nAlcoholism\nAlexander technique\nAlfalfa\nAlisma\nAllergies\nAllium cepa\nAloe\nAlpha-hydroxy\nAlzheimer’s disease\nAmino acids\nAndrographis\nAndrostenedione\nAnemarrhena\nAnemia\nAngelica root\nAngina\nAnise\nAnkylosing spondylitis\nAnorexia nervosa\nAnthroposophical medicine\nAntioxidants\nAnxiety\nBates method\nBayberry\nBedsores\nBedwetting\nBee pollen\nBehavioral optometry\nBehavioral therapy'),
 Document(id='fb813229-3209-4d4a-9331-31840c33144d', metadata={'author'

In [95]:
# from langchain_openai import OpenAI
# llm=OpenAI(temperature=0.4,max_tokens=500)
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0.4,
    max_tokens=500,
    model_name="llama3-70b-8192"  # or another supported Groq model
)


In [96]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [97]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [98]:
prompt=ChatPromptTemplate.from_messages([
    ('system',system_prompt),
    ('human',"{input}")
])

In [99]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [106]:
response=rag_chain.invoke({"input":"what is Diabetes mellitus?"})
print(response["answer"])

Diabetes mellitus is a chronic disease that causes serious health complications, including renal failure, heart disease, stroke, and blindness. It occurs when the body's cells stop responding to insulin, preventing glucose in the blood from being absorbed into the cells. This leads to symptoms such as frequent urination, tiredness, excessive thirst, and hunger.
